In [1]:
import pandas as pd

train_dataset = pd.read_csv('/content/drive/MyDrive/Datavidia/train_dataset.csv', index_col=0)
Y_training_df = train_dataset["main-label"]
# Drop 'label' column
X_training_df = train_dataset.drop(labels = ['main-label','main-id'],axis = 1) 
print('Y_training shape', Y_training_df.shape)
print('X_training shape', X_training_df.shape) 

Y_training shape (14856,)
X_training shape (14856, 15408)


In [19]:
import numpy as np

X_training_array = X_training_df.values
X_training = np.reshape(X_training_array,(14856,48,107,3))
Y_training = Y_training_df.values

In [20]:
# Split the train and the validation set for the fitting
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X_training, Y_training, test_size = 0.3, random_state=2)
print("x_train shape",X_train.shape)
print("x_val shape",X_val.shape)
print("y_train shape",Y_train.shape)
print("y_val shape",Y_val.shape)

x_train shape (10399, 48, 107, 3)
x_val shape (4457, 48, 107, 3)
y_train shape (10399,)
y_val shape (4457,)


In [22]:
from tensorflow.keras.applications import *
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras import optimizers
#MobileNet
base_model = MobileNetV2(input_shape=(48,107,3),include_top=False,classifier_activation=None)
x = base_model.output
x = Flatten()(x)
predictions = Dense(1, activation='sigmoid')(x) 
model = Model(inputs=base_model.input, outputs=predictions)
opt = optimizers.Adam(lr=0.001)

model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
print(model.summary())

74842112/74836368 [==============================] - 1s 0us/step
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 48, 107, 3)] 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 54, 113, 3)   0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 24, 54, 64)   9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 24, 54, 64)   256         conv1/conv[0][0]                 
___________________________

In [23]:
epochs = 20  # for better result increase the epochs
batch_size = 16

# Fit the model
history = model.fit(X_train, Y_train, 
                    epochs = epochs, validation_data = (X_val,Y_val), batch_size=batch_size)

Epoch 1/20
650/650 [==============================] - 68s 76ms/step - loss: 0.4620 - accuracy: 0.8546 - val_loss: 0.3875 - val_accuracy: 0.8811
Epoch 2/20
650/650 [==============================] - 45s 69ms/step - loss: 0.3193 - accuracy: 0.8957 - val_loss: 3.6813 - val_accuracy: 0.8578
Epoch 3/20
650/650 [==============================] - 45s 70ms/step - loss: 0.2728 - accuracy: 0.9019 - val_loss: 0.2890 - val_accuracy: 0.9046
Epoch 4/20
650/650 [==============================] - 45s 69ms/step - loss: 0.2700 - accuracy: 0.9106 - val_loss: 0.4472 - val_accuracy: 0.9006
Epoch 5/20
650/650 [==============================] - 45s 70ms/step - loss: 0.2526 - accuracy: 0.9100 - val_loss: 0.3763 - val_accuracy: 0.9073
Epoch 6/20
650/650 [==============================] - 45s 70ms/step - loss: 0.2184 - accuracy: 0.9193 - val_loss: 0.3339 - val_accuracy: 0.9080
Epoch 7/20
650/650 [==============================] - ETA: 0s - loss: 0.1918 - accuracy: 0.9321

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt

# Plot the loss and accuracy curves for training and validation 
plt.plot(history.history['val_loss'], color='b', label="validation loss", linestyle=':')
plt.plot(history.history['loss'], color='b', label="loss", linestyle='-')
plt.plot(history.history['val_accuracy'], color='r', label="validation accuracy", linestyle=':')
plt.plot(history.history['accuracy'], color='r', label="accuracy", linestyle='-')
plt.title("Test Loss")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

### Test Section

In [ ]:
import pandas as pd

test_dataset = pd.read_csv('/content/drive/MyDrive/Datavidia/test_dataset.csv', index_col=0)
# Drop 'label' column
X_testing_df = test_dataset.drop(labels = ['main-id'],axis = 1)
print('X_test shape', X_testing_df.shape) 

In [ ]:
import numpy as np

X_testing_array = X_testing_df.values
X_testing = np.reshape(X_testing_array,(3714,48,107,3))

In [ ]:
from copy import deepcopy
import numpy as np

def binary_classifier(label):
  temp = deepcopy(label)
  for i in temp:
    if i[0] > 0.5:
      i[0] = 1
    else:
      i[0] = 0
  return temp

In [ ]:
# confusion matrix
import seaborn as sns
# Predict the values from the validation dataset
Y_pred = model.predict(X_testing)
Y_pred_advanced = binary_classifier(Y_pred).astype(int)
Y_pred_advanced = Y_pred_advanced.reshape(-1)
print(Y_pred_advanced)

In [ ]:
temp1 = test_dataset['main-id'].values
test_result = list(zip(list(temp1),list(Y_pred_advanced)))
test_result_df = pd.DataFrame(test_result,columns=['review_id','category'])
test_result_df

In [ ]:
test_result_df.to_csv('/content/drive/MyDrive/Datavidia/test_result-ResNet50V2.csv', index=False)